<a href="https://colab.research.google.com/github/CristianS1988/GooSL/blob/main/Laboratorio3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Generador de texto


In [1]:
import tensorflow as tf
import requests

# Descargar el texto del libro
url = "https://www.gutenberg.org/files/47631/47631-0.txt"  # Reemplaza esta URL con la del libro que desees
response = requests.get(url)
book_text = response.text

# Tokenización
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([book_text])
total_words = len(tokenizer.word_index) + 1

# Crear secuencias de n-gramas
sequences = []
lines = book_text.split('\n')
for line in lines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

# Padding de secuencias
max_sequence_length = max([len(seq) for seq in sequences])
sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_sequence_length, padding="pre")

# Separar datos de entrada y etiquetas
input_sequences, labels = sequences[:, :-1], sequences[:, -1]
labels = tf.keras.utils.to_categorical(labels, num_classes=total_words)

# Modelo básico
modelo = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_length - 1),
    tf.keras.layers.LSTM(150),
    tf.keras.layers.Dense(total_words, activation="softmax")
])

# Compilar
modelo.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer="adam", metrics=["accuracy"])

# Entrenamiento
modelo.fit(x=input_sequences, y=labels, epochs=20)


Epoch 1/20
24/24 [==============================] - 2s 26ms/step - loss: 5.4236 - accuracy: 0.0365
Epoch 2/20
24/24 [==============================] - 1s 25ms/step - loss: 5.0140 - accuracy: 0.0755
Epoch 3/20
24/24 [==============================] - 1s 26ms/step - loss: 4.9210 - accuracy: 0.0885
Epoch 4/20
24/24 [==============================] - 1s 26ms/step - loss: 4.8227 - accuracy: 0.1107
Epoch 5/20
24/24 [==============================] - 1s 25ms/step - loss: 4.6842 - accuracy: 0.1276
Epoch 6/20
24/24 [==============================] - 1s 25ms/step - loss: 4.4860 - accuracy: 0.1419
Epoch 7/20
24/24 [==============================] - 1s 27ms/step - loss: 4.2316 - accuracy: 0.1849
Epoch 8/20
24/24 [==============================] - 1s 25ms/step - loss: 3.9971 - accuracy: 0.2240
Epoch 9/20
24/24 [==============================] - 1s 40ms/step - loss: 3.7695 - accuracy: 0.2474
Epoch 10/20
24/24 [==============================] - 1s 33ms/step - loss: 3.5611 - accuracy: 0.2682
Epoch 11/

### Modelo mejorado

In [2]:
# Modelo mejorado
modelo_mejorado = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_length - 1),
    tf.keras.layers.LSTM(150, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(total_words/2, activation="relu"),
    tf.keras.layers.Dense(total_words, activation="softmax")
])

# Compilar
modelo_mejorado.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer="adam", metrics=["accuracy"])

# Entrenamiento
modelo_mejorado.fit(x=input_sequences, y=labels, epochs=30)


Epoch 1/30
24/24 [==============================] - 4s 42ms/step - loss: 5.4385 - accuracy: 0.0443
Epoch 2/30
24/24 [==============================] - 1s 59ms/step - loss: 5.0618 - accuracy: 0.0729
Epoch 3/30
24/24 [==============================] - 1s 50ms/step - loss: 4.9743 - accuracy: 0.0729
Epoch 4/30
24/24 [==============================] - 1s 43ms/step - loss: 4.9435 - accuracy: 0.0729
Epoch 5/30
24/24 [==============================] - 1s 43ms/step - loss: 4.8885 - accuracy: 0.0729
Epoch 6/30
24/24 [==============================] - 1s 44ms/step - loss: 4.7757 - accuracy: 0.0781
Epoch 7/30
24/24 [==============================] - 1s 44ms/step - loss: 4.6208 - accuracy: 0.0742
Epoch 8/30
24/24 [==============================] - 1s 45ms/step - loss: 4.4482 - accuracy: 0.0924
Epoch 9/30
24/24 [==============================] - 1s 43ms/step - loss: 4.2657 - accuracy: 0.0964
Epoch 10/30
24/24 [==============================] - 1s 44ms/step - loss: 4.1220 - accuracy: 0.1237
Epoch 11/

### Ajuste de Hiperparámetros

In [5]:
!pip install keras-tuner
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(total_words, hp.Int('embedding_dim', 50, 150, 10), input_length=max_sequence_length - 1))
    model.add(tf.keras.layers.LSTM(hp.Int('lstm_units', 50, 200, 10)))
    model.add(tf.keras.layers.Dense(total_words, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2
)

tuner.search(x=input_sequences, y=labels, epochs=5, validation_split=0.2)


Reloading Tuner from ./untitled_project/tuner0.json


### Validación y Pruebas

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(input_sequences, labels, test_size=0.2)
modelo.fit(x=x_train, y=y_train, validation_data=(x_val, y_val), epochs=20)


Epoch 1/20
20/20 [==============================] - 2s 49ms/step - loss: 1.8624 - accuracy: 0.5717 - val_loss: 1.8691 - val_accuracy: 0.5714
Epoch 2/20
20/20 [==============================] - 1s 31ms/step - loss: 1.7419 - accuracy: 0.6140 - val_loss: 2.0004 - val_accuracy: 0.5325
Epoch 3/20
20/20 [==============================] - 1s 58ms/step - loss: 1.6457 - accuracy: 0.6156 - val_loss: 2.0994 - val_accuracy: 0.5000
Epoch 4/20
20/20 [==============================] - 1s 45ms/step - loss: 1.5486 - accuracy: 0.6564 - val_loss: 2.1790 - val_accuracy: 0.4675
Epoch 5/20
20/20 [==============================] - 1s 32ms/step - loss: 1.4533 - accuracy: 0.6694 - val_loss: 2.2493 - val_accuracy: 0.4805
Epoch 6/20
20/20 [==============================] - 1s 32ms/step - loss: 1.3651 - accuracy: 0.6971 - val_loss: 2.2968 - val_accuracy: 0.4610
Epoch 7/20
20/20 [==============================] - 1s 32ms/step - loss: 1.2894 - accuracy: 0.7134 - val_loss: 2.3610 - val_accuracy: 0.4675
Epoch 8/20
20

### Visualizaciones

In [7]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs')
modelo.fit(x=input_sequences, y=labels, epochs=20, callbacks=[tensorboard_callback])


Epoch 1/20
24/24 [==============================] - 2s 46ms/step - loss: 1.1093 - accuracy: 0.7760
Epoch 2/20
24/24 [==============================] - 1s 27ms/step - loss: 0.9933 - accuracy: 0.7799
Epoch 3/20
24/24 [==============================] - 1s 27ms/step - loss: 0.9060 - accuracy: 0.7904
Epoch 4/20
24/24 [==============================] - 1s 27ms/step - loss: 0.8321 - accuracy: 0.8008
Epoch 5/20
24/24 [==============================] - 1s 27ms/step - loss: 0.7815 - accuracy: 0.8216
Epoch 6/20
24/24 [==============================] - 1s 27ms/step - loss: 0.7412 - accuracy: 0.8320
Epoch 7/20
24/24 [==============================] - 1s 30ms/step - loss: 0.7044 - accuracy: 0.8451
Epoch 8/20
24/24 [==============================] - 1s 28ms/step - loss: 0.6748 - accuracy: 0.8516
Epoch 9/20
24/24 [==============================] - 1s 28ms/step - loss: 0.6454 - accuracy: 0.8542
Epoch 10/20
24/24 [==============================] - 1s 27ms/step - loss: 0.6223 - accuracy: 0.8529
Epoch 11/

### Regularización

In [8]:
from tensorflow.keras.regularizers import l2

modelo = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_length - 1),
    tf.keras.layers.LSTM(150, recurrent_dropout=0.2, kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dense(total_words, activation="softmax")
])


In [9]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)


{'embedding_dim': 80, 'lstm_units': 180}


In [10]:
best_model = tuner.get_best_models(num_models=1)[0]


In [11]:
tuner.results_summary()


Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
embedding_dim: 80
lstm_units: 180
Score: 0.10389610379934311

Trial 0 summary
Hyperparameters:
embedding_dim: 90
lstm_units: 160
Score: 0.10064934939146042

Trial 3 summary
Hyperparameters:
embedding_dim: 120
lstm_units: 170
Score: 0.08116883039474487

Trial 4 summary
Hyperparameters:
embedding_dim: 130
lstm_units: 60
Score: 0.07467532530426979

Trial 2 summary
Hyperparameters:
embedding_dim: 130
lstm_units: 50
Score: 0.07467532530426979


In [12]:
best_model.fit(x=input_sequences, y=labels, epochs=50)


Epoch 1/50
24/24 [==============================] - 2s 30ms/step - loss: 4.8210 - accuracy: 0.1211
Epoch 2/50
24/24 [==============================] - 1s 29ms/step - loss: 4.5688 - accuracy: 0.1406
Epoch 3/50
24/24 [==============================] - 1s 29ms/step - loss: 4.2957 - accuracy: 0.1693
Epoch 4/50
24/24 [==============================] - 1s 30ms/step - loss: 4.0666 - accuracy: 0.2135
Epoch 5/50
24/24 [==============================] - 1s 29ms/step - loss: 3.8170 - accuracy: 0.2396
Epoch 6/50
24/24 [==============================] - 1s 30ms/step - loss: 3.5904 - accuracy: 0.2630
Epoch 7/50
24/24 [==============================] - 1s 34ms/step - loss: 3.3920 - accuracy: 0.2839
Epoch 8/50
24/24 [==============================] - 1s 57ms/step - loss: 3.1849 - accuracy: 0.3099
Epoch 9/50
24/24 [==============================] - 1s 30ms/step - loss: 2.9929 - accuracy: 0.3203
Epoch 10/50
24/24 [==============================] - 1s 30ms/step - loss: 2.8196 - accuracy: 0.3555
Epoch 11/

NameError: ignored

<Figure size 1200x600 with 0 Axes>